In [22]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn import preprocessing
from scipy.stats import skew, boxcox
from IPython.core.pylabtools import figsize
import xgboost as xgb
%matplotlib inline

from os import path
to_filename = lambda name: path.join("..", "data", "allstate", name +".csv")

import seaborn as sns
sns.set_style("whitegrid")

In [23]:
train = pd.read_csv(to_filename("train"), index_col=0)
test = pd.read_csv(to_filename("test"), index_col=0)
print("shape: train {}, test {}".format(train.shape, test.shape))
print(train.head(2))

shape: train (188318, 131), test (125546, 130)
   cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9 cat10   ...        cont6  \
id                                                      ...                
1     A    B    A    B    A    A    A    A    B     A   ...     0.718367   
2     A    B    A    A    A    A    A    A    B     B   ...     0.438917   

       cont7    cont8    cont9   cont10    cont11    cont12    cont13  \
id                                                                      
1   0.335060  0.30260  0.67135  0.83510  0.569745  0.594646  0.822493   
2   0.436585  0.60087  0.35127  0.43919  0.338312  0.366307  0.611431   

      cont14     loss  
id                     
1   0.714843  2213.18  
2   0.304496  1283.60  

[2 rows x 131 columns]


In [24]:
response = np.log(train.loss)

mean_resp = np.mean(response)
std_resp = np.std(response)
response = (response - mean_resp) / std_resp


def restore_pred1(y):
    return np.exp(y)

def restore_pred(y):
    return np.exp(y * std_resp + mean_resp)

In [25]:
cat_features = [col for col in train.columns if col.startswith("cat")]
print("Categorical columns:", cat_features)

('Categorical columns:', ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21', 'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28', 'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35', 'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42', 'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49', 'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56', 'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63', 'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70', 'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77', 'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84', 'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91', 'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98', 'cat99', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat107', 'cat108', 'c

In [ ]:
# Categorical features preprocessing
# Method 1: Encoding categorical features into int
for col in cat_features:
    encd = preprocessing.LabelEncoder()
    encd.fit(train[col].value_counts().index.union(test[col].value_counts().index))
    train[col] = encd.transform(train[col])
    test[col] = encd.transform(test[col])

In [26]:
# Method 2: Using cardinal features for categorical features
col = cat_features[0]
test_col = train[col][:10].copy()
for col in cat_features:
    key_map = response.groupby(train[col]).mean().to_dict()
    train[col] = train[col].replace(key_map)
    for k in set(test[col].value_counts().index).difference(key_map.keys()):
        key_map[k] = np.NAN
    test[col] = test[col].replace(key_map)

In [27]:
# preprocess numerical features
num_features = [col for col in train.columns if col.startswith("cont")]
print("Numerical columns:", num_features)

('Numerical columns:', ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14'])


In [ ]:
# Method 1: Standard Scaler
for col in num_features:
    sc = preprocessing.StandardScaler()
    train[col] = sc.fit_transform(train[[col]])
    test[col] = sc.transform(test[[col]])

In [28]:
# study the skewness in the numerical features
skewed_feats = pd.concat([train[num_features], test[num_features]]).apply(lambda x: skew(x.dropna()))
print("Skew in numeric features:", skewed_feats)

('Skew in numeric features:', cont1     0.513205
cont2    -0.311146
cont3    -0.007023
cont4     0.417559
cont5     0.679610
cont6     0.458413
cont7     0.825889
cont8     0.673237
cont9     1.067247
cont10    0.352116
cont11    0.281139
cont12    0.291997
cont13    0.376138
cont14    0.250673
dtype: float64)


In [30]:
# Method 2: Box-Cox transformation when numerical feature skewness > .25
for feat in skewed_feats[skewed_feats > 0.25].index:
    train[feat], lam = boxcox(train[feat] + 1.)
    test[feat], lam = boxcox(test[feat] + 1.)

In [31]:
dtrain = xgb.DMatrix(train.drop("loss", 1), response)
dtest = xgb.DMatrix(test)

In [32]:
params = {'objective':"reg:linear", 'silent': True, 'max_depth': 7, 'min_child_weight': 1,
          'colsample_bytree': .7, "subsample": 1., 'eta': 0.1, 'eval_metric':'mae',# "n_estimators": 20,
          "gamma": 0.5, "lambda": 0.8, "silent": 1}

In [17]:
cvresult = xgb.cv(params, dtrain, nfold=3, num_boost_round=50)
print(cvresult)

    test-mae-mean  test-mae-std  train-mae-mean  train-mae-std
0        0.791689      0.003139        0.790803       0.001235
1        0.711323      0.003149        0.709352       0.002114
2        0.656812      0.002042        0.653948       0.001794
3        0.620731      0.002414        0.616826       0.000628
4        0.596710      0.002187        0.591768       0.000454
5        0.580212      0.002258        0.574466       0.000412
6        0.568539      0.002131        0.561919       0.000628
7        0.559974      0.001806        0.552459       0.000483
8        0.553738      0.001513        0.545400       0.000547
9        0.548844      0.001217        0.539716       0.000382
10       0.545042      0.000931        0.534977       0.000428
11       0.541749      0.000705        0.530983       0.000334
12       0.539042      0.000683        0.527467       0.000154
13       0.536925      0.000308        0.524565       0.000420
14       0.534990      0.000514        0.521948       0

In [ ]:
cvresult[["test-mae-mean", "train-mae-mean"]].plot()

In [ ]:
cvresult[["test-mae-mean", "train-mae-mean"]].plot()

In [36]:
folds = 3

pred_test = 0.
pred_train = 0.
restored_pred_train = 0.
restored_pred_test = 0.

kf = KFold(n_splits=folds)
kf.split(train)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    train_pd_ind = train.index[train_index]
    test_pd_ind = train.index[test_index]
    train_part, test_part = train.ix[train_pd_ind], train.ix[test_pd_ind]
    
    dtrain_part = xgb.DMatrix(train_part.drop("loss", 1), response[train_pd_ind])
    dtest_part = xgb.DMatrix(test_part.drop("loss", 1), response[test_pd_ind])
    params['seed'] = i * 5 + 100
    clf = xgb.train(params, dtrain_part, num_boost_round=300,
                    evals=[(dtrain_part, "train"), (dtest_part, "eval")])
    
    this_pred_train = clf.predict(dtrain, ntree_limit=clf.best_ntree_limit)
    print("mae for all train",i, mean_absolute_error(train.loss, restore_pred(this_pred_train)))
    
    pred_train += this_pred_train
    restored_pred_train += restore_pred(this_pred_train)
    
    this_pred_test = clf.predict(dtest, ntree_limit=clf.best_ntree_limit)
    pred_test += this_pred_test
    restored_pred_test += restore_pred(this_pred_test)
    

[0]	train-mae:0.862118	eval-mae:0.863862
[1]	train-mae:0.817106	eval-mae:0.81905
[2]	train-mae:0.778025	eval-mae:0.780266
[3]	train-mae:0.744741	eval-mae:0.747218
[4]	train-mae:0.715918	eval-mae:0.718649
[5]	train-mae:0.692098	eval-mae:0.694921
[6]	train-mae:0.67084	eval-mae:0.673868
[7]	train-mae:0.65255	eval-mae:0.655659
[8]	train-mae:0.637208	eval-mae:0.640569
[9]	train-mae:0.624119	eval-mae:0.627853
[10]	train-mae:0.612469	eval-mae:0.61644
[11]	train-mae:0.602162	eval-mae:0.606215
[12]	train-mae:0.593057	eval-mae:0.597417
[13]	train-mae:0.58527	eval-mae:0.589938
[14]	train-mae:0.578482	eval-mae:0.583359
[15]	train-mae:0.572265	eval-mae:0.577558
[16]	train-mae:0.56706	eval-mae:0.57257
[17]	train-mae:0.562433	eval-mae:0.568329
[18]	train-mae:0.557944	eval-mae:0.564207
[19]	train-mae:0.554005	eval-mae:0.560586
[20]	train-mae:0.550594	eval-mae:0.557621
[21]	train-mae:0.547647	eval-mae:0.554989
[22]	train-mae:0.544768	eval-mae:0.552466
[23]	train-mae:0.54236	eval-mae:0.550468
[24]	train

In [37]:
print("mae final restore after", mean_absolute_error(train.loss, restore_pred(pred_train / folds)))
print("mae final restore before", mean_absolute_error(train.loss, restored_pred_train / folds))

('mae final restore after', 1043.7366157892552)
('mae final restore before', 1042.7750201282063)


In [38]:
import datetime
result = pd.DataFrame({"id": test.index, "loss": restored_pred_test / folds})
result.to_csv("result_restored_before{:%Y%m%d%H}.csv".format(datetime.datetime.now()), index=None)

# Using XGBRegressor and important features

In [ ]:
from sklearn.grid_search import GridSearchCV
from xgboost import XGBRegressor

In [ ]:
params_reg = dict(params)
params_reg.pop("eta")
params_reg.pop('eval_metric')
params_reg.pop('lambda')

In [ ]:
reg = XGBRegressor(**params_reg)
reg.fit(train.drop("loss", 1), train.loss)

In [ ]:
train_predprob = reg.score()

In [ ]:
reg_booster = reg.booster()

In [ ]:
figsize(18, 5)
feat_imp = pd.Series(reg.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')

In [ ]:
important_features = list(feat_imp[feat_imp > 4].index)
print("important features:", important_features)

In [ ]:
dtrain_imp = xgb.DMatrix(train[important_features], train.loss)
cvresult = xgb.cv(params, dtrain_imp, nfold=4, num_boost_round=50)
print(cvresult)

In [ ]:
params2 = {'base_score': 0.1, 'colsample_bytree': 0.9,
 'eta': 0.3,
 'eval_metric': 'mae',
 'max_depth': 7,
 'min_child_weight': 3,
 'n_estimators': 10,
 'objective': 'reg:linear',
 'seed': 1,
 'silent': True}
regb = xgb.train(params2, dtrain_imp, num_boost_round=50, evals=[(dtrain_imp, "train")])

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator=reg, 
 param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4, iid=False, cv=5)

In [ ]:
gsearch1.fit(train.drop("loss", 1), train.loss)